In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["WANDB_DISABLED"] = "true"

import sys
sys.path = [
    p for p in sys.path
    if p not in ['/home/jxm3/research/prompting/imodelsX', '/home/jxm3/research/prompting/tree-prompt']
]
sys.path.append('/home/jxm3/research/retrieval/inversion')

In [2]:
import aliases

os.environ["TOKENIZERS_MULTIPROCESSING"] = "True"

# inv_trainer = aliases.load_trainer_from_alias("openai_msmarco__msl128__100epoch")
corr_experiment, corr_trainer = aliases.load_experiment_and_trainer_from_alias(
    "openai_msmarco__msl128__100epoch__correct"
)
inv_trainer = corr_trainer.inversion_trainer
corr_trainer.precompute_hypotheses()
corr_trainer.model.eval()
print()

loading alias openai_msmarco__msl128__100epoch__correct from /home/jxm3/research/retrieval/inversion/saves/d6ec9d5838a4ad3daeba636e5378a8a0...
> checkpoint: /home/jxm3/research/retrieval/inversion/saves/d6ec9d5838a4ad3daeba636e5378a8a0/checkpoint-384000
Corrector encoder noise level 0.001
loading alias openai_msmarco__msl128__100epoch from /home/jxm3/research/retrieval/inversion/saves/f9abd65db4c4823264b133816d08612f...
Overwriting max sequence length from 128 to 128
Overwriting use_less_data from 1000000 to 1000000
> checkpoint: /home/jxm3/research/retrieval/inversion/saves/f9abd65db4c4823264b133816d08612f/checkpoint-194000


/home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Loading datasets with TOKENIZERS_PARALLELISM = False


[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
	Decoded output shape ->  torch.Size([1, 86])
	Decoded output -> And the trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls, trolls, was brilliant, and smooty.
================ End trainer sanity check ================
Froze 353779584 params from model type <class 'models.inversion.InversionModel'>


[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
	Decoded output shape ->  torch.Size([1, 64])
	Decoded output -> Was brilliant, and witty, And witty, and witty, and witty, and witty, and witty, and witty, witty, witty, witty, witty, witty, witty.
================ End trainer sanity check ================
Loading hypotheses from path /home/jxm3/research/retrieval/inversion/saves/f9abd65db4c4823264b133816d08612f/9d4a4d4b36da188a6e9dcb9736262823/0ed77465b20070e5_hypotheses.cache
Loading hypotheses from path /home/jxm3/research/retrieval/inversion/saves/f9abd65db4c4823264b133816d08612f/9d4a4d4b36da188a6e9dcb9736262823/fa5c492104fd5235_hypotheses.cache
Loading hypotheses from path /home/jxm3/research/retrieval/inversion/saves/f9abd65db4c4823264b133816d08612f/9d4a4d4b36da188a6e9dcb9736262823/927a7aa89af4e080_hypotheses.cache
Loading 

In [3]:
corr_trainer.return_best_hypothesis = False
corr_trainer.args.per_device_eval_batch_size = 128
corr_trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}
corr_trainer.num_gen_recursive_steps = 1
metrics = corr_trainer.evaluate(
    # eval_dataset=corr_trainer.train_dataset.select(range(100)),
    eval_dataset=corr_trainer.eval_dataset["msmarco"].select(range(100)),
    metric_key_prefix="eval_test",
)
print(metrics)

/home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[pred] [Submitting Floor Plan: The proposed use permit will require the separate parking space for each existing. drive room and driveway.] The dimensions of the proposed use will be shown on the building dimensions sheet as a complete, symmetrical size.
[true] *The use of the driveway as the required parking spaces will require a separate submittal for a Use Permit. [ ] Floor plan (dimensioned): Show overall building size, existing and proposed room locations, existing and. proposed new window(s)/door(s) size/location(s).



[pred] Our INSPIRED Seville map is fully illustrated with a map of each street, allowing you to explore the hidden treasures of the city in an intuitive manner. Our Seville map also includes a map of the city's central square, a map of the city's streets and a map of the city's parks and gardens.reyvale, Spainâs hidden treasures, and the quaint streets of Seville, are all accessible to anyone with a keen sense of location and a keen sense of direction.
[true] Our 

In [9]:
corr_trainer.return_best_hypothesis = False
corr_trainer.args.per_device_eval_batch_size = 128
corr_trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}
corr_trainer.num_gen_recursive_steps = 1
metrics = corr_trainer.evaluate(
    eval_dataset=corr_trainer.train_dataset.select(range(100)),
    # eval_dataset=corr_trainer.eval_dataset["msmarco"].select(range(100)),
    metric_key_prefix="eval_test",
)
print(metrics)

[pred] Examples of Binary Acids. Binary acids begin with the name of the element, such as H2O (Hydrochloric Acid), H3O (Hydrochloric Acid), H2O (Hydrochloric Acid), and H3O (Hydrochloric Acid). These acids are followed by one other element, such as a nonmetal, such as a metal, such as a tetrafluorine or a tetrahydroxide. The two nonmetals are named in pairs, like te
[true] Examples: HF, HCl, HBr, HI, H2O, H2S. The names of binary acids begin with hydro-followed by the name of the other element modified to end with-ic (like in hydro chlor ic acid). Binary Acids are one of two classes of acids, the second being the Ternary Acids. The Ternary acids either consist of a hydrogen, oxygen and one other nonmetal element called the oxoacids (like HNO3), or they consist of hydrogen and two other nonmetal elements,



[pred] Co:Writer should now be in the beginning of sentences. Co:Writer will send out sentences as you type. You will be able to concentrate on one word at a time. As you type, Co:W

In [15]:
import datasets
import random
import torch
train_data_subset = corr_trainer.eval_dataset["msmarco"].select(range(200)).to_dict()
# random.shuffle(train_data_subset['frozen_embeddings'])
perm = torch.randperm(200)
train_data_subset['hypothesis_input_ids'] = [train_data_subset['hypothesis_input_ids'][i] for i in perm]
train_data_subset['hypothesis_attention_mask'] = [train_data_subset['hypothesis_attention_mask'][i] for i in perm]
random.shuffle(train_data_subset['hypothesis_embedding'])
train_data_subset = datasets.Dataset.from_dict(train_data_subset)
train_data_subset.set_format("pt")

In [16]:
corr_trainer.return_best_hypothesis = False
corr_trainer.args.per_device_eval_batch_size = 128
corr_trainer.gen_kwargs = {
    "early_stopping": False,
    "num_beams": 1,
    "do_sample": False,
    "no_repeat_ngram_size": 0,
}
corr_trainer.num_gen_recursive_steps = 1
metrics = corr_trainer.evaluate(
    eval_dataset=train_data_subset,
    # eval_dataset=corr_trainer.eval_dataset["msmarco"].select(range(100)),
    metric_key_prefix="eval_test",
)
print(metrics)

[pred] This use of the parking lot will require a separate permit for each additional parking space. [See also: Floor Plan Use of the Parking Lot.]. _____________________________________________________________________________________________________________________________________________________________________________________________________________. The existing subdivision is divided into distinct types of void spaces, called insets, that are affixed to the wall.
[true] *The use of the driveway as the required parking spaces will require a separate submittal for a Use Permit. [ ] Floor plan (dimensioned): Show overall building size, existing and proposed room locations, existing and. proposed new window(s)/door(s) size/location(s).



[pred] Our SEREA map is illuminated with a hidden map of each street in the city. The map is able to reveal the hidden streets of each city, allowing you to make an accurate exploration of the city's major cities.ourtiland, Seville, and Seville-Antwe

In [8]:
corr_trainer.train_dataset[1]

{'input_ids': tensor([  638,    10, 24965,    49,   225,   230,    36,    16,     8,    21,
            15,  9232,     5, 10237,  4892,   324,  2319,    10,   148,    56,
          1431, 16513,    80,    44,     3,     9,    97,    28,   638,    10,
         24965,    49,     5,   282,    25,  1731,    12,   686,     3,     9,
          1448,     6,   638,    10, 24965,    49,   704,  3382,    15,     7,
            38,    12,   125,     8,  1448,    19,     6,     3,   390,    30,
             8,  5487,    25,   686,    26,    11,   119,  2580,     5,     1]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'labels': tensor([  638,    10, 24965,    49,   225,   230,    36,    16,     8,    21,
            15,  9232,     5, 10237,  4892,   324,  2319,    10,   148,    56,
       

In [7]:
corr_trainer.eval_dataset['msmarco'][0]

{'input_ids': tensor([ 1429,   634,   169,    13,     8, 16860,    38,     8,   831,  3078,
          4856,    56,  1457,     3,     9,  2450,  4237,  1947,    21,     3,
             9,  2048,  1915,  1538,     5,   784,     3,   908, 12324,   515,
            41, 31987,    15,    26,    61,    10,  3111,  1879,   740,   812,
             6,  1895,    11,  4382,   562,  3248,     6,  1895,    11,     5,
          4382,   126,  2034,   599,     7,    61,    87, 11968,   599,     7,
            61,   812,    87, 14836,   599,     7,   137,     1]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'labels': tensor([ 1429,   634,   169,    13,     8, 16860,    38,     8,   831,  3078,
          4856,    56,  1457,     3,     9,  2450,  4237,  1947,    21,     3,
             9,  2048,  191

In [42]:
d2.keys() <= d1.keys()

True

In [44]:
for k in d2.keys():
    print((d1[k] == d1[k]).all())

tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)


In [53]:
inputs = {k: v[None].to(corr_trainer.args.device) for k,v in d1.items()}
try:
    frozen_embeddings1 = inputs["frozen_embeddings"]
    ################################################################################
    # # print("temp: starting from best hypothesis")
    # hypothesis_input_ids = inputs["best_hypothesis_input_ids"]
    # hypothesis_embedding = inputs["best_hypothesis_embedding"]
    # hypothesis_attention_mask = (hypothesis_input_ids != self.model.encoder_decoder.config.pad_token_id).int()
    ################################################################################
    hypothesis_input_ids1 = inputs["hypothesis_input_ids"]
    hypothesis_attention_mask1 = inputs["hypothesis_attention_mask"]
    hypothesis_embedding1 = inputs["hypothesis_embedding"]
    ################################################################################
except KeyError:
    (
        frozen_embeddings1,
        hypothesis_input_ids1,
        hypothesis_attention_mask1,
        hypothesis_embedding1,
    ) = corr_trainer_no_precompute._get_hypothesis_uncached(inputs=inputs)


In [52]:
inputs = {k: v[None].to(corr_trainer.args.device) for k,v in d2.items()}
try:
    frozen_embeddings2 = inputs["frozen_embeddings"]
    ################################################################################
    # # print("temp: starting from best hypothesis")
    # hypothesis_input_ids = inputs["best_hypothesis_input_ids"]
    # hypothesis_embedding = inputs["best_hypothesis_embedding"]
    # hypothesis_attention_mask = (hypothesis_input_ids != self.model.encoder_decoder.config.pad_token_id).int()
    ################################################################################
    hypothesis_input_ids2 = inputs["hypothesis_input_ids"]
    hypothesis_attention_mask2 = inputs["hypothesis_attention_mask"]
    hypothesis_embedding2 = inputs["hypothesis_embedding"]
    ################################################################################
except KeyError:
    (
        frozen_embeddings2,
        hypothesis_input_ids2,
        hypothesis_attention_mask2,
        hypothesis_embedding2,
    ) = corr_trainer_not_precomputed._get_hypothesis_uncached(inputs=inputs)


In [55]:
(frozen_embeddings1 == frozen_embeddings2).all()

tensor(True, device='cuda:0')

In [56]:
(hypothesis_embedding1 == hypothesis_embedding2).all()

tensor(False, device='cuda:0')

In [57]:
hypothesis_input_ids1

tensor([[    0, 19119,    13,     3, 19990,   447, 18802,     7,     5, 17239,
           524,   322,   447, 15215,  1731,    28,     8,   564,    13,     8,
             3,    23,  4554, 12771,     6,   224,    38,     3,     9,   835,
          1162,    41,   566,   357,   667,   201,     3,     9,   835,  1162,
            41,   566,   357,   667,   201,    42,     3,     9,   835,  1162,
            41,   566,   357,   667,   137, 17239,   729,   447, 15215,    33,
          2348,    57,   192,   119,     3,    23,  4554, 18042,     6,   224,
            38,     3,     9,   835,  1162,    41,   566,   357,   667,   201,
            11,     3,     9,   835,  1162,    41,   566,   357,   667,   137,
         17239,   729,   447, 15215,    33,    80,    13,     8,   386,  2287,
            13,     3,    23,  4554, 18042,     5,   204,  5388,    18,  3493,
          5405,     6,   224,    38,     3,     9,     3,  3828,  5255,     6,
            33,   718, 14865,     3,    23,  4554, 1

In [58]:
hypothesis_input_ids2

tensor([[    0, 19119,    13, 18802,   447, 18802,     7,   560,     3, 17097,
             6,   454,   357,   667,     6,   454,   357,   667,     6,    11,
           454,   357,   667,     5, 17239,   729,   447, 18802,     7,  1731,
            28,     8,   564,    13,     8,     3,    23,  4554, 12771,    41,
           566,    18,    61,  2348,    57,     8,   564,    13,     8,     3,
            23,  4554, 12771,    41,   566,    18,   137, 17239,   729,   447,
         15215,    33,    80,    13,   192,  2287,    13,     3,    23,  4554,
         18042,     6,     8,   119,   192,   271,     3, 15979, 15215,    41,
           566,    18,    61,    11,     3, 15979, 15215,    41,   279,    18,
           137,     3,  8739,     3,     9,     3,    17,    15,  1313,    88,
            26,  4900,  3562,     6,     3,     9,     3, 15979,  3562,    19,
             3,     9,     3,    17,    15,  1313,    88,    26,  4900, 12771,
            28,     3,     9,     3,    17,    15,  

In [63]:
corr_trainer.gen_kwargs

{'early_stopping': False,
 'num_beams': 1,
 'do_sample': False,
 'no_repeat_ngram_size': 0}

In [66]:
gen_kwargs = {
    'early_stopping': False,
    'num_beams': 1,
    'do_sample': False,
    'no_repeat_ngram_size': 0,
    'min_length': 1,
    'max_length': 129,
}
corr_trainer.inversion_trainer.model.generate(inputs=inputs, generation_kwargs=gen_kwargs)

tensor([[    0, 19119,    13, 18802,   447, 18802,     7,   560,     3, 17097,
             6,   454,   357,   667,     6,   454,   357,   667,     6,    11,
           454,   357,   667,     5, 17239,   729,   447, 18802,     7,  1731,
            28,     8,   564,    13,     8,     3,    23,  4554, 12771,    41,
           566,    18,    61,  2348,    57,     8,   564,    13,     8,     3,
            23,  4554, 12771,    41,   566,    18,   137, 17239,   729,   447,
         15215,    33,    80,    13,   192,  2287,    13,     3,    23,  4554,
         18042,     6,     8,   119,   192,   271,     3, 15979, 15215,    41,
           566,    18,    61,    11,     3, 15979, 15215,    41,   279,    18,
           137,     3,  8739,     3,     9,     3,    17,    15,  1313,    88,
            26,  4900,  3562,     6,     3,     9,     3, 15979,  3562,    19,
             3,     9,     3,    17,    15,  1313,    88,    26,  4900, 12771,
            28,     3,     9,     3,    17,    15,  